In [1]:
import json 
import pandas as pd

with open("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/text_embeddings.json", "r") as f:
    data = json.load(f)

    embeddings = data['embeddings']
    embeddings = {int(key): value for key, value in embeddings.items()}

    text_idxs = data['text_idxs']
    text_idxs = {key: int(value) for key, value in text_idxs.items()} 
    
    pairs_ranking = data['pairs_ranking']
    pairs_ranking = {tuple(json.loads(key)): value for key, value in pairs_ranking.items()}

    pairs = list(pairs_ranking.keys())
    pairs = [(embeddings[pair[0]], embeddings[pair[1]]) for pair in pairs]
    labels = list(pairs_ranking.values())

We have neutral-neutral pairs, and dataset with the hawkish/dovish augmentations.

Then, if doc1 > doc2 (more hawkish) -> doc1.hawkish  > doc2, doc2.dovish., doc1 > doc2.dovish
If doc1 < doc2 (more dovish) -> doc1.dovish < doc2, doc1 < doc2.hawkish, doc1.dovish < doc2.hawkih


In [27]:
qa_df = pd.read_csv("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/qa_data_labeled.csv")

with open("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/train_pairs_ranking.json", "r") as f:
    train_pairs_ranking = json.load(f)
    train_pairs_ranking = {tuple(json.loads(key)): value for key, value in train_pairs_ranking.items()}

with open("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/test_pairs_ranking.json", "r") as f:
    test_pairs_ranking = json.load(f)
    test_pairs_ranking = {tuple(json.loads(key)): value for key, value in test_pairs_ranking.items()}


def generate_pairs_ranking(qa_df, pairs_ranking):
    result_pairs_ranking = {}

    for pair, value in pairs_ranking.items():
        try:
            if value == 1:
                higher_text = pair[0]
                lower_text = pair[1]
            else:
                lower_text = pair[0]
                higher_text = pair[1]
                
            higher_text_date = qa_df.loc[qa_df['text'] == higher_text]['date'].values[0]
            lower_text_date = qa_df.loc[qa_df['text'] == lower_text]['date'].values[0]

            higher_text_hawkish = qa_df.loc[(qa_df['date'] == higher_text_date) & (qa_df['shift'] == 'hawkish')]['text'].values[0]
            higher_text_dovish = qa_df.loc[(qa_df['date'] == higher_text_date) & (qa_df['shift'] == 'dovish')]['text'].values[0]

            lower_text_hawkish = qa_df.loc[(qa_df['date'] == lower_text_date) & (qa_df['shift'] == 'hawkish')]['text'].values[0]
            lower_text_dovish = qa_df.loc[(qa_df['date'] == lower_text_date) & (qa_df['shift'] == 'dovish')]['text'].values[0]

            #Infered Pairs for higher and lower texts
            result_pairs_ranking[(higher_text_hawkish, lower_text)] =  abs(value)
            result_pairs_ranking[(higher_text, lower_text_dovish)] =  abs(value)
            result_pairs_ranking[(higher_text_hawkish, lower_text_dovish)] =  1

            #Original Pairs from the triplets 
            result_pairs_ranking[(higher_text_hawkish, higher_text)] =  0.5
            result_pairs_ranking[(higher_text, higher_text_dovish)] =  0.5
            result_pairs_ranking[(higher_text_hawkish, higher_text_dovish)] =  1

            result_pairs_ranking[(lower_text_hawkish, lower_text)] =  0.5
            result_pairs_ranking[(lower_text, lower_text_dovish)] =  0.5
            result_pairs_ranking[(lower_text_hawkish, lower_text_dovish)] =  1
        except BaseException as exc: #We dodge exceptions for blocks where some augmentations are missing
            pass 

    return result_pairs_ranking

In [30]:
train_pairs_ranking = generate_pairs_ranking(qa_df, train_pairs_ranking)
test_pairs_ranking = generate_pairs_ranking(qa_df, test_pairs_ranking)

train_pairs_ranking = {json.dumps(key): value for key, value in train_pairs_ranking.items()}
test_pairs_ranking = {json.dumps(key): value for key, value in test_pairs_ranking.items()}

with open("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/train_pairs_ranking_linked.json", "w") as f:
    json.dump(train_pairs_ranking, f)

with open("/Users/dzz1th/Job/mgi/Soroka/data/qa_data/test_pairs_ranking_linked.json", "w") as f:
    json.dump(test_pairs_ranking, f)

In [32]:
len(test_pairs_ranking)

1158